In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader
import torcheval

from sklearn.decomposition import TruncatedSVD
import pandas as pd
import joblib

In [19]:
df = pd.read_csv("data_transformation/chipseq/updated_histone_mods_and_GA_factors.feature_matrix.1kb_res.txt", sep="\t")
df['seq'] = df['seq'].map(lambda x: x.upper())
df['gene_labels'] = df['gene_labels'].astype(int)
df['mre_labels'] = df['mre_labels'].astype(int)
df.head()

,bin1_id,chr,start,end,clamp,gaf,h3k27ac,h3k27me3,h3k36me3,h3k4me1,h3k4me2,h3k4me3,h3k9me3,h4k16ac,psq,seq,gene_labels,mre_labels
0,0,chr2L,0,1000,0.011287,0.026900,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,30.045745,CGACAATGCACGACAGAGGAAGCAGAACAGATATTTAGATTGCCTC...,0,1
1,1,chr2L,1000,2000,0.011801,0.029785,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,30.008160,CGTATGCGAGAGTAGTGCCAACATATTGTGCTCTTTGATTTTTTGG...,0,1
2,2,chr2L,2000,3000,0.011536,0.029358,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,30.077650,ATTCAAGTTGCCGCTAATCAGAAATAAATTCATTGCAACGTTAAAT...,0,1
3,3,chr2L,3000,4000,0.011079,0.030652,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,40.028765,AACATATTGTGCTAATGAGTGCCTCTCGTTCTCTGTCTTATATTAC...,0,1
4,4,chr2L,4000,5000,0.011053,0.032013,0.0,0.0,20.0032,0.0,0.0,0.0,0.0,0.0,40.014130,GCAGCGGAGATATTTAGATTGCCTATTAAATATGATCGCGTATGCG...,0,1


In [20]:
# Imports
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import matthews_corrcoef, f1_score, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

from accelerate.test_utils.testing import get_backend
device, _, _ = get_backend()

## Gene Task Fine-Tuning

In [5]:
num_labels_gene = 3

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("InstaDeepAI/nucleotide-transformer-v2-50m-multi-species", num_labels=num_labels_gene)
model = model.to(device)

Loading InstaDeepAI/nucleotide-transformer-v2-50m-multi-species requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/InstaDeepAI/nucleotide-transformer-v2-50m-multi-species. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading InstaDeepAI/nucleotide-transformer-v2-50m-multi-species requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/InstaDeepAI/nucleotide-transformer-v2-50m-multi-species. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


Some weights of the model checkpoint at InstaDeepAI/nucleotide-transformer-v2-50m-multi-species were not used when initializing EsmForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-v2-50m-multi-species and are newly initialized: ['classifier.out_proj.bias',

In [21]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, inference_mode=False, r=1, lora_alpha= 32, lora_dropout=0.1, target_modules= ["query", "value"],
    modules_to_save=["intermediate"] # modules that are not frozen and updated during the training
)

from peft import get_peft_model

lora_classifier = get_peft_model(model, peft_config) # transform our classifier into a peft model
lora_classifier.print_trainable_parameters()
lora_classifier.to(device) # Put the model on the GPU

trainable params: 25,454,595 || all params: 79,253,191 || trainable%: 32.118069542461704


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): EsmForSequenceClassification(
      (esm): EsmModel(
        (embeddings): EsmEmbeddings(
          (word_embeddings): Embedding(4107, 512, padding_idx=1)
          (dropout): Dropout(p=0.0, inplace=False)
          (position_embeddings): Embedding(2050, 512, padding_idx=1)
        )
        (encoder): EsmEncoder(
          (layer): ModuleList(
            (0): EsmLayer(
              (attention): EsmAttention(
                (self): EsmSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=512, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=1, bias=False)
                    )
                    (lora_B): ModuleDict(
                

In [41]:
from datasets import load_dataset, Dataset

X, y = df['seq'], df['gene_labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

# validation = 0.25 * 0.8 = 0.2; train size = 0.6 overall
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=1, stratify=y_train)

# Gene Label Dataset dataset
ds_train = Dataset.from_dict({"data": X_train,'labels':y_train})
ds_val = Dataset.from_dict({"data": X_val,'labels':y_val})
ds_test = Dataset.from_dict({"data": X_test,'labels':y_test})

In [42]:
len(X_train) / 16, len(X_val) / 16, len(X_test) / 16

(6018.6875, 859.875, 1719.6875)

In [43]:
tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-v2-50m-multi-species")

def tokenize_function(examples):
    outputs = tokenizer(examples["data"])
    return outputs

In [44]:
tokenized_datasets_train_promoter = ds_train.map(
    tokenize_function,
    batched=True,
    remove_columns=["data"],
)
tokenized_datasets_validation_promoter = ds_val.map(
    tokenize_function,
    batched=True,
    remove_columns=["data"],
)
tokenized_datasets_test_promoter = ds_test.map(
    tokenize_function,
    batched=True,
    remove_columns=["data"],
)

Map:   0%|          | 0/96299 [00:00<?, ? examples/s]

Map:   0%|          | 0/13758 [00:00<?, ? examples/s]

Map:   0%|          | 0/27515 [00:00<?, ? examples/s]

In [60]:
batch_size = 16
model_name='nucleotide-transformer'
args_promoter = TrainingArguments(
    f"models/{model_name}-50m",
    overwrite_output_dir=True,
    remove_unused_columns=False,
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps= 4,
    per_device_eval_batch_size= 64,
    num_train_epochs= 2,
    logging_steps=1000,
    save_steps=1000,
    load_best_model_at_end=True,  # Keep the best model according to the evaluation
    metric_for_best_model="f1_score",
    label_names=["labels"],
    dataloader_drop_last=True,
    max_steps= 5000,
    report_to="wandb"
)

import torch.nn.functional as F
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=-1)
    raw_out = torch.Tensor(eval_pred.predictions)
    logits = F.softmax(raw_out, dim=-1).numpy()
    n_classes = logits.shape[-1] if logits.ndim > 1 else 2
    
    labels = eval_pred.label_ids
    ohe_labels = np.eye(n_classes, dtype=int)[labels]
    
    try:
        r={
            'f1_score': f1_score(labels, predictions, average="macro"),
            'accuracy': np.mean(labels == predictions),
            'avg_auroc': roc_auc_score(labels, logits, multi_class='ovr', average='macro'),
            'avg_auprc': average_precision_score(ohe_labels, logits)
        }
    except Exception as e:
        print("Error on the following:")
        print(f"logits: {logits}, {type(logits)}")
        print(f"predictions: {predictions}, {type(predictions)}")
        print(f"labels: {labels}, {type(labels)}")
        raise ValueError("error produced from above vectors.")
    return r

In [61]:
import os
import wandb

wandb.init(project="bindgps_seq_nt-finetune", name="50m", reinit=True)

eval/accuracy,▅▁███
eval/avg_auprc,▁▇▇██
eval/avg_auroc,▁▇▇██
eval/f1_score,▃█▂▁▃
eval/loss,█▃▁▁▁
eval/runtime,▁█▇▅▅
eval/samples_per_second,█▁▂▄▄
eval/steps_per_second,█▁▂▃▄
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
train/learning_rate,█▆▅▃▁


In [62]:
trainer = Trainer(
    model.to(device),
    args_promoter,
    train_dataset= tokenized_datasets_train_promoter,
    eval_dataset= tokenized_datasets_validation_promoter,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/oscar/home/pmahable/my_env/lib64/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


In [32]:
wandb.finish()

train/epoch,▁
train/global_step,▁
train/learning_rate,▁
train/loss,▁
train/epoch,0.08
train/global_step,1000
train/learning_rate,0.0004
train/loss,0.9192


In [ ]:
curve_evaluation_f1_score =[[a['step'],a['eval_f1_score']] for a in trainer.state.log_history if 'eval_f1_score' in a.keys()]
eval_f1_score = [c[1] for c in curve_evaluation_f1_score]
steps = [c[0] for c in curve_evaluation_f1_score]

In [ ]:
plt.plot(steps, eval_f1_score, 'b', label='Validation F1 score')
plt.title('Validation F1 score for promoter prediction')
plt.xlabel('Number of training steps performed')
plt.ylabel('Validation F1 score')
plt.legend()
plt.show()

In [ ]:
print(f"F1 score on the test dataset: {trainer.predict(tokenized_datasets_test_promoter).metrics['test_f1_score']}")